In [44]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer


import re


from nltk.corpus import stopwords
import pickle

from tqdm import tqdm
import os

In [45]:
project_df = pd.read_csv('train_data.csv', nrows=5000)
resource_df = pd.read_csv('resources.csv')

In [46]:
project_df.head()

,Unnamed: 0,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved
0,160221,p253737,c90749f5d961ff158d4b4d1e7dc665fc,Mrs.,IN,2016-12-05 13:43:57,Grades PreK-2,Literacy & Language,"ESL, Literacy",Educational Support for English Learners at Home,My students are English learners that are work...,"\""The limits of your language are the limits o...",NaN,NaN,My students need opportunities to practice beg...,0,0
1,140945,p258326,897464ce9ddc600bced1151f324dd63a,Mr.,FL,2016-10-25 09:22:10,Grades 6-8,"History & Civics, Health & Sports","Civics & Government, Team Sports",Wanted: Projector for Hungry Learners,Our students arrive to our school eager to lea...,The projector we need for our school is very c...,NaN,NaN,My students need a projector to help with view...,7,1
2,21895,p182444,3465aaf82da834c0582ebd0ef8040ca0,Ms.,AZ,2016-08-31 12:03:56,Grades 6-8,Health & Sports,"Health & Wellness, Team Sports",Soccer Equipment for AWESOME Middle School Stu...,"\r\n\""True champions aren't always the ones th...",The students on the campus come to school know...,NaN,NaN,"My students need shine guards, athletic socks,...",1,0
3,45,p246581,f3cb9bffbba169bef1a77b243e620b60,Mrs.,KY,2016-10-06 21:16:17,Grades PreK-2,"Literacy & Language, Math & Science","Literacy, Mathematics",Techie Kindergarteners,I work at a unique school filled with both ESL...,My students live in high poverty conditions wi...,NaN,NaN,My students need to engage in Reading and Math...,4,1
4,172407,p104768,be1f7507a41f8479dc06f047086a39ec,Mrs.,TX,2016-07-11 01:10:09,Grades PreK-2,Math & Science,Mathematics,Interactive Math Tools,Our second grade classroom next year will be m...,"For many students, math is a subject that does...",NaN,NaN,My students need hands on practice in mathemat...,1,1


In [47]:
project_df['project_grade_category']

0       Grades PreK-2
1          Grades 6-8
2          Grades 6-8
3       Grades PreK-2
4       Grades PreK-2
            ...      
4995    Grades PreK-2
4996    Grades PreK-2
4997    Grades PreK-2
4998       Grades 6-8
4999       Grades 6-8
Name: project_grade_category, Length: 5000, dtype: object

In [48]:
#we need to remove the spaces, replace the '-' with '_' and convert all the letters to small of project_grade_category
project_df['project_grade_category'] = project_df['project_grade_category'].str.replace(' ', '_')
project_df['project_grade_category'] = project_df['project_grade_category'].str.replace('-', '_')
project_df['project_grade_category'] = project_df['project_grade_category'].str.lower()

In [49]:
project_df['project_grade_category']

0       grades_prek_2
1          grades_6_8
2          grades_6_8
3       grades_prek_2
4       grades_prek_2
            ...      
4995    grades_prek_2
4996    grades_prek_2
4997    grades_prek_2
4998       grades_6_8
4999       grades_6_8
Name: project_grade_category, Length: 5000, dtype: object

In [50]:
project_df['project_subject_categories'].unique()

array(['Literacy & Language', 'History & Civics, Health & Sports',
       'Health & Sports', 'Literacy & Language, Math & Science',
       'Math & Science', 'Literacy & Language, Special Needs',
       'Literacy & Language, Applied Learning', 'Special Needs',
       'Math & Science, Literacy & Language', 'Applied Learning',
       'Math & Science, Special Needs', 'Music & The Arts',
       'History & Civics', 'Health & Sports, Literacy & Language',
       'Literacy & Language, Music & The Arts', 'Warmth, Care & Hunger',
       'Math & Science, History & Civics',
       'Applied Learning, Literacy & Language',
       'Applied Learning, Special Needs',
       'Literacy & Language, History & Civics',
       'Applied Learning, Health & Sports',
       'History & Civics, Literacy & Language',
       'Health & Sports, Special Needs',
       'Applied Learning, Math & Science',
       'Math & Science, Music & The Arts',
       'Health & Sports, Applied Learning',
       'History & Civics, Musi

In [51]:
project_df['project_subject_categories'] = project_df['project_subject_categories'].str.replace(' The', '_')
project_df['project_subject_categories'] = project_df['project_subject_categories'].str.replace('&', '_')
project_df['project_subject_categories'] = project_df['project_subject_categories'].str.replace(',', '_')
project_df['project_subject_categories'] = project_df['project_subject_categories'].str.replace(' ', '')
project_df['project_subject_categories'] = project_df['project_subject_categories'].str.lower()
project_df['project_subject_categories'].unique()

array(['literacy_language', 'history_civics_health_sports',
       'health_sports', 'literacy_language_math_science', 'math_science',
       'literacy_language_specialneeds',
       'literacy_language_appliedlearning', 'specialneeds',
       'math_science_literacy_language', 'appliedlearning',
       'math_science_specialneeds', 'music__arts', 'history_civics',
       'health_sports_literacy_language', 'literacy_language_music__arts',
       'warmth_care_hunger', 'math_science_history_civics',
       'appliedlearning_literacy_language',
       'appliedlearning_specialneeds', 'literacy_language_history_civics',
       'appliedlearning_health_sports',
       'history_civics_literacy_language', 'health_sports_specialneeds',
       'appliedlearning_math_science', 'math_science_music__arts',
       'health_sports_appliedlearning', 'history_civics_music__arts',
       'math_science_appliedlearning', 'music__arts_history_civics',
       'appliedlearning_music__arts', 'history_civics_math_scie

In [52]:
project_df['project_subject_categories'].value_counts()

literacy_language                       1067
math_science                             795
literacy_language_math_science           679
health_sports                            509
music__arts                              233
literacy_language_specialneeds           207
appliedlearning                          164
specialneeds                             162
math_science_literacy_language           101
appliedlearning_literacy_language         97
appliedlearning_specialneeds              80
math_science_specialneeds                 80
literacy_language_music__arts             79
math_science_music__arts                  76
history_civics_literacy_language          65
history_civics                            63
health_sports_specialneeds                57
warmth_care_hunger                        53
math_science_appliedlearning              52
appliedlearning_math_science              44
health_sports_literacy_language           40
literacy_language_history_civics          40
appliedlea

In [53]:
project_df['teacher_prefix'].unique()

array(['Mrs.', 'Mr.', 'Ms.', 'Teacher'], dtype=object)

In [54]:
#Remove '.' 
#convert all the chars to small
project_df['teacher_prefix']=project_df['teacher_prefix'].str.replace('.','')
project_df['teacher_prefix']=project_df['teacher_prefix'].str.lower()
project_df['teacher_prefix'].unique()

array(['mrs', 'mr', 'ms', 'teacher'], dtype=object)

In [55]:
#let's do what we did with subject categories with subcategories
project_df['project_subject_subcategories'] = project_df['project_subject_subcategories'].str.replace(' The', '_')
project_df['project_subject_subcategories'] = project_df['project_subject_subcategories'].str.replace('&', '_')
project_df['project_subject_subcategories'] = project_df['project_subject_subcategories'].str.replace(',', '_')
project_df['project_subject_subcategories'] = project_df['project_subject_subcategories'].str.replace(' ', '')
project_df['project_subject_subcategories'] = project_df['project_subject_subcategories'].str.lower()
project_df['project_subject_subcategories'].unique()

array(['esl_literacy', 'civics_government_teamsports',
       'health_wellness_teamsports', 'literacy_mathematics',
       'mathematics', 'literature_writing_specialneeds',
       'literacy_specialneeds', 'health_wellness',
       'literacy_literature_writing', 'literacy',
       'literacy_parentinvolvement',
       'environmentalscience_health_lifescience', 'specialneeds',
       'appliedsciences_literature_writing', 'earlydevelopment',
       'health_lifescience_specialneeds', 'music',
       'appliedsciences_mathematics', 'foreignlanguages_mathematics',
       'literacy_other', 'health_lifescience_literacy',
       'economics_financialliteracy', 'literature_writing', 'teamsports',
       'literature_writing_mathematics', 'health_wellness_literacy',
       'gym_fitness_health_wellness', 'literacy_visualarts',
       'warmth_care_hunger', 'mathematics_socialsciences', 'visualarts',
       'college_careerprep_literature_writing', 'environmentalscience',
       'appliedsciences_health_l

In [56]:
#school states are in capital, let's convert them to small
project_df['school_state'] = project_df['school_state'].str.lower()
project_df['school_state'].unique()

array(['in', 'fl', 'az', 'ky', 'tx', 'ct', 'ga', 'sc', 'nc', 'ca', 'ny',
       'ok', 'ma', 'nv', 'oh', 'pa', 'al', 'la', 'va', 'ar', 'wa', 'wv',
       'id', 'tn', 'ms', 'co', 'ut', 'il', 'mi', 'hi', 'ia', 'ri', 'nj',
       'mo', 'de', 'mn', 'me', 'wy', 'nd', 'or', 'ak', 'md', 'wi', 'sd',
       'ne', 'nm', 'dc', 'ks', 'mt', 'nh', 'vt'], dtype=object)

In [57]:
project_df['project_title']

0        Educational Support for English Learners at Home
1                   Wanted: Projector for Hungry Learners
2       Soccer Equipment for AWESOME Middle School Stu...
3                                  Techie Kindergarteners
4                                  Interactive Math Tools
                              ...                        
4995                     Chromebooks to Maximize Success!
4996                                   ilearn in literacy
4997                                      Read in Comfort
4998                   Read, Baby, Read: Favorite Titles!
4999                            SOCK IT OUT IN BASKETBALL
Name: project_title, Length: 5000, dtype: object

In [58]:
# https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python/47091490#47091490
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [59]:
# https://gist.github.com/sebleier/554280

stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]

In [60]:
project_df['project_title'].values[9]

'Just For the Love of Reading--\\r\\nPure Pleasure'

In [61]:
from tqdm import tqdm

def preprocess_text(text):
    text_preprocess = []
    for sentence in tqdm(text):
        senten = decontracted(sentence)
        senten = senten.replace('\\r', ' ')
        senten = senten.replace('\\n', ' ')
        senten = senten.replace('\\"', ' ')
        senten = re.sub('[^A-Za-z0-9]+', ' ', senten)
        senten = ' '.join(e for e in senten.split() if e.lower() not in stopwords)
        text_preprocess.append(senten)
    return text_preprocess

In [62]:
preprocessed_project_title = preprocess_text(project_df['project_title'].values)

100%|██████████| 5000/5000 [00:00<00:00, 23316.33it/s]


In [63]:
preprocessed_project_title[9]

'Love Reading Pure Pleasure'

In [64]:
#Processing essay data:

In [66]:
# merge two column text dataframe: 
project_df["essay"] = project_df["project_essay_1"].map(str) +\
                        project_df["project_essay_2"].map(str) + \
                        project_df["project_essay_3"].map(str) + \
                        project_df["project_essay_4"].map(str)

In [67]:
preprocessed_essays = preprocess_text(project_df['essay'].values)

100%|██████████| 5000/5000 [00:04<00:00, 1223.49it/s]


In [89]:
project_df['essay'] = preprocessed_essays

In [90]:
resource_df.head(5)

,id,description,quantity,price
0,p233245,LC652 - Lakeshore Double-Space Mobile Drying Rack,1,149.00
1,p069063,Bouncy Bands for Desks (Blue support pipes),3,14.95
2,p069063,Cory Stories: A Kid's Book About Living With Adhd,1,8.45
3,p069063,"Dixon Ticonderoga Wood-Cased #2 HB Pencils, Bo...",2,13.59
4,p069063,EDUCATIONAL INSIGHTS FLUORESCENT LIGHT FILTERS...,3,24.95


In [70]:
resource_df[resource_df['id'] == 'p000001']

,id,description,quantity,price
414179,p000001,"Cap Barbell 300 Pound Olympic Set, Grey",2,261.08
414180,p000001,Cap Barbell Power Rack Exercise Stand,2,89.00
414181,p000001,Marcy SB-10510 Flat Bench,1,85.49
414182,p000001,ProSource Puzzle Exercise Mat High Quality EVA...,2,23.99


In [71]:
price_data = resource_df.groupby('id').agg({'price':'sum', 'quantity':'sum'}).reset_index()

In [74]:
project_df = project_df.merge(price_data, on='id', how='left')

In [75]:
project_df

,Unnamed: 0,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved,essay,price,quantity
0,160221,p253737,c90749f5d961ff158d4b4d1e7dc665fc,mrs,in,2016-12-05 13:43:57,grades_prek_2,literacy_language,esl_literacy,Educational Support for English Learners at Home,My students are English learners that are work...,"\""The limits of your language are the limits o...",NaN,NaN,My students need opportunities to practice beg...,0,0,My students are English learners that are work...,154.60,23
1,140945,p258326,897464ce9ddc600bced1151f324dd63a,mr,fl,2016-10-25 09:22:10,grades_6_8,history_civics_health_sports,civics_government_teamsports,Wanted: Projector for Hungry Learners,Our students arrive to our school eager to lea...,The projector we need for our school is very c...,NaN,NaN,My students need a projector to help with view...,7,1,Our students arrive to our school eager to lea...,299.00,1
2,21895,p182444,3465aaf82da834c0582ebd0ef8040ca0,ms,az,2016-08-31 12:03:56,grades_6_8,health_sports,health_wellness_teamsports,Soccer Equipment for AWESOME Middle School Stu...,"\r\n\""True champions aren't always the ones th...",The students on the campus come to school know...,NaN,NaN,"My students need shine guards, athletic socks,...",1,0,"\r\n\""True champions aren't always the ones th...",516.85,22
3,45,p246581,f3cb9bffbba169bef1a77b243e620b60,mrs,ky,2016-10-06 21:16:17,grades_prek_2,literacy_language_math_science,literacy_mathematics,Techie Kindergarteners,I work at a unique school filled with both ESL...,My students live in high poverty conditions wi...,NaN,NaN,My students need to engage in Reading and Math...,4,1,I work at a unique school filled with both ESL...,232.90,4
4,172407,p104768,be1f7507a41f8479dc06f047086a39ec,mrs,tx,2016-07-11 01:10:09,grades_prek_2,math_science,mathematics,Interactive Math Tools,Our second grade classroom next year will be m...,"For many students, math is a subject that does...",NaN,NaN,My students need hands on practice in mathemat...,1,1,Our second grade classroom next year will be m...,67.98,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,121624,p182497,c8de6793242dc67e94fc7a44d095a77e,mrs,ca,2016-10-14 12:08:46,grades_prek_2,literacy_language_specialneeds,literature_writing_specialneeds,Chromebooks to Maximize Success!,"Noise from the fluorescent lightbulbs pulse, s...",My students have autism and are included in ge...,NaN,NaN,My students need 2 chromebooks to type papers ...,1,1,"Noise from the fluorescent lightbulbs pulse, s...",277.42,2
4996,159460,p186132,fbf06153496bbfcb76d3fc40a8dffb74,ms,nc,2016-04-27 16:14:26,grades_prek_2,literacy_language_music__arts,literacy_music,ilearn in literacy,Who doesn't love to sit down and read a good b...,I have 23 students that love to get curled up ...,"When you walk into my room, it is very rare th...",Your donations to our listening center with iP...,My students need 4 ipod shuffles for our liste...,5,1,Who doesn't love to sit down and read a good b...,201.07,4
4997,173241,p057102,2ea3c9aadf301783d55623d2d6d8efa6,mrs,sd,2016-10-14 14:37:18,grades_prek_2,literacy_language,literacy,Read in Comfort,"We live in a small community in South Dakota, ...",Practice makes perfect they always say. I wan...,NaN,NaN,My students need a variety of bean bag chairs ...,35,0,"We live in a small community in South Dakota, ...",115.29,11
4998,2688,p066753,2b3ae89a719c2b8f7d2311668ec94881,ms,la,2016-07-24 15:46:30,grades_6_8,literacy_language_specialneeds,literacy_specialneeds,"Read, Baby, Read: Favorite Titles!","My students are READING BEASTS! Last year, we...",I teach 60 students (and support the other 150...,NaN,NaN,My students need more copies our favorite book...,7,1,"My students are READING

In [77]:
project_df.shape

(5000, 20)

In [79]:
project_df['price'].head()

0    154.60
1    299.00
2    516.85
3    232.90
4     67.98
Name: price, dtype: float64

In [81]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(project_df['price'].values.reshape(-1, 1))
project_df['std_price']=scaler.transform(project_df['price'].values.reshape(-1, 1) )

In [82]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(project_df['price'].values.reshape(-1, 1))
project_df['nrm_price']=scaler.transform(project_df['price'].values.reshape(-1, 1))

In [83]:
project_df.head()

,Unnamed: 0,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,...,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved,essay,price,quantity,std_price,nrm_price
0,160221,p253737,c90749f5d961ff158d4b4d1e7dc665fc,mrs,in,2016-12-05 13:43:57,grades_prek_2,literacy_language,esl_literacy,Educational Support for English Learners at Home,...,NaN,NaN,My students need opportunities to practice beg...,0,0,My students are English learners that are work...,154.60,23,-0.393708,0.015320
1,140945,p258326,897464ce9ddc600bced1151f324dd63a,mr,fl,2016-10-25 09:22:10,grades_6_8,history_civics_health_sports,civics_government_teamsports,Wanted: Projector for Hungry Learners,...,NaN,NaN,My students need a projector to help with view...,7,1,Our students arrive to our school eager to lea...,299.00,1,-0.010053,0.029763
2,21895,p182444,3465aaf82da834c0582ebd0ef8040ca0,ms,az,2016-08-31 12:03:56,grades_6_8,health_sports,health_wellness_teamsports,Soccer Equipment for AWESOME Middle School Stu...,...,NaN,NaN,"My students need shine guards, athletic socks,...",1,0,"\r\n\""True champions aren't always the ones th...",516.85,22,0.568751,0.051554
3,45,p246581,f3cb9bffbba169bef1a77b243e620b60,mrs,ky,2016-10-06 21:16:17,grades_prek_2,literacy_language_math_science,literacy_mathematics,Techie Kindergarteners,...,NaN,NaN,My students need to engage in Reading and Math...,4,1,I work at a unique school filled with both ESL...,232.90,4,-0.185673,0.023152
4,172407,p104768,be1f7507a41f8479dc06f047086a39ec,mrs,tx,2016-07-11 01:10:09,grades_prek_2,math_science,mathematics,Interactive Math Tools,...,NaN,NaN,My students need hands on practice in mathemat...,1,1,Our second grade classroom next year will be m...,67.98,4,-0.623847,0.006656


In [87]:
preprocessed_data = project_df[['school_state', 'teacher_prefix', 'project_grade_category',
       'teacher_number_of_previously_posted_projects', 'project_is_approved',
       'project_subject_categories', 'project_subject_subcategories', 'essay', 'price']]

In [88]:
preprocessed_data.head()

,school_state,teacher_prefix,project_grade_category,teacher_number_of_previously_posted_projects,project_is_approved,project_subject_categories,project_subject_subcategories,essay,price
0,in,mrs,grades_prek_2,0,0,literacy_language,esl_literacy,My students are English learners that are work...,154.60
1,fl,mr,grades_6_8,7,1,history_civics_health_sports,civics_government_teamsports,Our students arrive to our school eager to lea...,299.00
2,az,ms,grades_6_8,1,0,health_sports,health_wellness_teamsports,"\r\n\""True champions aren't always the ones th...",516.85
3,ky,mrs,grades_prek_2,4,1,literacy_language_math_science,literacy_mathematics,I work at a unique school filled with both ESL...,232.90
4,tx,mrs,grades_prek_2,1,1,math_science,mathematics,Our second grade classroom next year will be m...,67.98
